In [1]:
import re

In [4]:
def split_dml(doc):
    '''
    input: file in many sql sentenses
    output: list of sql sentenses
    '''
    pass

def make_sql_token(sql_str):
    # remove the /*  */ comments
    xsql = re.sub(r"/\*[^*]*\*+(?:[^*/][^*]*\*+)*/", "", sql_str)
    
    # remove whole line -- and # comments
    lines = [line for line in xsql.splitlines() if not re.match("^\s*(--|#)", line)]
    
    # remove trailing -- and # comments    
    xsql = "".join([re.split("--|#", line)[0] for line in lines])

    # Make tokens
    xsql = xsql.lower()
    tokens = re.split(r"[\s)(;]+", xsql)
    
    # remove stop-words    
    #tokens.remove('')

    xdml_type = tokens[0]
    
    return xdml_type, tokens


def extract_tbl_select(tokens):
    # scan the tokens. if we see a FROM or JOIN, we set the get_next
    # flag, and grab the next one (unless it's SELECT).

    result = set()
    get_next = False
    
    for tok in tokens:
        if get_next:
            if tok not in ["", "select"]:
                result.add(tok)
            get_next = False
        get_next = tok in ["from", "join"]

    return result

In [6]:
'''
DML_TYPE, TBL_LIST
'''

r = open('../../data/sql/single_select.txt', mode='r', encoding='utf-8')
doc = r.read()
print(doc)
dml_type, sql_token = make_sql_token(doc)
print('\n\npure_sql',dml_type,sql_token,sep='\n')

tbl_set = extract_tbl_select(sql_token)
print('\n\ntbl_set', tbl_set,sep='\n')

/* 이것은 설명 */
			SELECT 	R.SEQ, R.CHARGE_TYPE, R.GRP_NO, R.V_CD, R.V_NO
					-- A CHARGE
					-- A CHARGE는 GROUP별로 구한다.
					, ROW_NUMBER() OVER(PARTITION BY R.SEQ, R.GRP_NO ORDER BY AMT DESC) AS RN_PC_MAX
					, ROW_NUMBER() OVER(PARTITION BY R.SEQ, R.GRP_NO ORDER BY AMT ASC) 	AS RN_PC_MIN
					-- B CHARGE
					, ROW_NUMBER() OVER(PARTITION BY R.SEQ ORDER BY AMT DESC) 				AS RN_CG_MAX
					, ROW_NUMBER() OVER(PARTITION BY R.SEQ ORDER BY AMT ASC) 				AS RN_CG_MIN
			FROM	(	
				SELECT	X.SEQ, X.CHARGE_TYPE, X.V_CD, X.V_NO, X.GRP_NO
						-- 
						, SUM(ISNULL(X.TOT_AMT,0)) 	
								OVER(PARTITION BY X.SEQ, X.CHARGE_TYPE, X.GRP_NO, X.V_CD, X.V_NO) AS AMT
				FROM (
						SELECT	C.SEQ, D.V_CD, D.V_NO, CB.GRP_NO, D.P_CD
								, C.CHARGE_CD AS CHARGE_TYPE
                                /***
								-- 금액 구하기 
                                ***/
								, CASE 
									WHEN D.LCL_CURR_CD IN ('KRW','USD','CNY','JPY') THEN
										CASE
											WHEN D.S_CURR_CD = 'KRW' 	THEN D.STL_AM